# Crawler

Date: 2024/05/02-

## 前処理
- Beautiful SoapでPDFファイルのリンクリスト作成
- PyMuPDFで各PDFファイルのText抽出しSQLiteへ格納
- 同時にspaCyでNERを行い、その結果をSQLiteへ格納

```
Table "sources" <= 経産省、総務省、XX株式会社といった白書やIR資料を提供している団体名
base_url(primary key), homepage_url, org, doc, doc_url

Table "links" <= Beautiful Soapで抽出したPDFファイルのURl
id, path, title, fk(sources:base_url)

Table "texts" <= PyMuPDFで抽出したテキスト
fk(links:link_id), page, text

Table "named_entities" <= spaCyで抽出したNamed Entity
fk(links:link_id), page, named_entity, label
```

## APIサーバ (Flaskで実装)

#### /sources

\[{base_url: \<base_url\>, homepage_url: \<homepage_url\>, org: \<base_url\>, doc: \<doc\>, doc_url: \<doc_url\>},...\]

PDFファイル検索対象となる団体のリストを返信（今回は経済産業省のみ）。

#### /search?base_url=\<base_url\>&keywords=\<keywords\>

\[{link_id: \<link\>, title: \<title\>, page: \<page\>, text: \<text\>, spans: {keyword: \[\[\<start\>, \<end\>\],...\],...}\]

該当キーワードを含むPDFファイルのパスとページ番号を返信。base_urlを指定しない場合、全ての団体を検索対象とする。

#### /highlight?link_id=\<link_id\>&page=\<page\>&keywords=\<keywords\>&all_pages=\<all_pages\>

該当キーワードをハイライトしてPDFを返送。指定ページの前後で合計３ページ分を返信。

## Table: sources

In [25]:
# 2023年度(令和5年)白書
ORGS = [
    ["経済産業省",
     "通商白書",
     "https://www.meti.go.jp",
     "https://www.meti.go.jp",
     "https://www.meti.go.jp/report/tsuhaku2023/whitepaper_2023.html"],
    ["総務省",
     "情報通信白書",
     "https://www.soumu.go.jp",
     "https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf",
     "https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf/index.html"],
    ["防衛省",
     "防衛白書",
     "https://www.mod.go.jp/",
     "http://www.clearing.mod.go.jp/hakusho_data/2023/pdf",
     "http://www.clearing.mod.go.jp/hakusho_data/2023/pdf/index.html"]
]

In [2]:
import sqlite3

DB_PATH = 'database/search.db'

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS sources')
    cur.execute('CREATE TABLE sources (base_url TEXT PRIMARY KEY, homepage_url TEXT, org TEXT, doc TEXT, doc_url TEXT, UNIQUE(homepage_url, org, doc, doc_url))')
    for o in ORGS:
        org = o[0]
        doc = o[1]
        homepage_url =o[2]
        base_url = o[3]
        doc_url = o[4]
        cur.execute(f'INSERT INTO sources (base_url, homepage_url, org, doc, doc_url) VALUES ("{base_url}", "{homepage_url}", "{org}", "{doc}", "{doc_url}")')

## Table: links

PDFリンク抽出

In [3]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    base_urls = cur.execute('SELECT base_url FROM sources').fetchall()

base_urls

[('http://www.clearing.mod.go.jp/hakusho_data/2023/pdf',),
 ('https://www.meti.go.jp',),
 ('https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf',)]

## 各省庁向け準備

In [4]:
import requests
from bs4 import BeautifulSoup
import re

In [5]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS links')
    cur.execute('CREATE TABLE links (id INTEGER PRIMARY KEY AUTOINCREMENT, path TEXT, title TEXT, base_url TEXT, UNIQUE(path, title, base_url), FOREIGN KEY(base_url) REFERENCES sources(base_url))')

### 経済産業省　通商白書

In [6]:
url_meti = "https://www.meti.go.jp/report/tsuhaku2023/whitepaper_2023.html"

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    base_url = cur.execute('SELECT base_url FROM sources WHERE org="経済産業省" AND doc="通商白書"').fetchone()[0]

base_url

'https://www.meti.go.jp'

In [7]:
resp = requests.get(url_meti)
html_doc = resp.content.decode('utf-8')

In [8]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [9]:
chapter1 = [tag for tag in soup.find_all(string='第Ⅰ部　岐路に立たされる世界経済')]

In [10]:
all_a = chapter1[0].find_all_next("a", href=re.compile(r'^.*\d?-\d?-\d?\.pdf$'))

# [[url, title], ...]
links = [[a['href'], a.text] for a in all_a]

In [11]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    for path, title in links:
        cur.execute(f'INSERT INTO links (path, title, base_url) VALUES(?, ?, ?)', (path, title, base_url))

### 総務省　情報通信白書

In [12]:
url_soumu = "https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf/index.html"

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    base_url = cur.execute('SELECT base_url FROM sources WHERE org="総務省" AND doc="情報通信白書"').fetchone()[0]

base_url

'https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf'

In [13]:
resp = requests.get(url_soumu)

In [14]:
# Reference: https://stackoverflow.com/questions/7219361/python-and-beautifulsoup-encoding-issues
from bs4.dammit import EncodingDetector
html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
html_encoding

'shift_jis'

In [15]:
soup = BeautifulSoup(resp.content.decode(html_encoding))
all_a = soup.find_all('a', href=re.compile(r'^n\d\d00000\.pdf$'))
links = [[a['href'], a.text] for a in all_a]
sorted(links)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


[['n1100000.pdf', 'データ流通を支える通信インフラの高度化'],
 ['n1200000.pdf', 'データ流通とデジタルサービスの進展'],
 ['n2100000.pdf', '加速するデータ流通とデータ利活用'],
 ['n2200000.pdf', 'プラットフォーマーへのデータの集中'],
 ['n2300000.pdf', 'インターネット上での偽・誤情報の拡散等'],
 ['n3100000.pdf', 'データ流通・活用の新たな潮流'],
 ['n3200000.pdf', '豊かなデータ流通社会の実現に向けて'],
 ['n4100000.pdf', 'ICT産業の動向'],
 ['n4200000.pdf', '電気通信分野の動向'],
 ['n4300000.pdf', '放送・コンテンツ分野の動向'],
 ['n4400000.pdf', '我が国の電波の利用状況'],
 ['n4500000.pdf', '国内外におけるICT機器・端末関連の動向'],
 ['n4600000.pdf', 'プラットフォームの動向'],
 ['n4700000.pdf', 'ICTサービス及びコンテンツ・アプリケーションサービス市場の動向'],
 ['n4800000.pdf', 'データセンター市場及びクラウドサービス市場の動向'],
 ['n4900000.pdf', 'AIの動向'],
 ['n5100000.pdf', '総合的なICT政策の推進'],
 ['n5200000.pdf', '電気通信事業政策の動向'],
 ['n5300000.pdf', '電波政策の動向'],
 ['n5400000.pdf', '放送政策の動向'],
 ['n5500000.pdf', 'サイバーセキュリティ政策の動向'],
 ['n5600000.pdf', 'ICT利活用の推進'],
 ['n5700000.pdf', 'ICT技術政策の動向'],
 ['n5800000.pdf', 'ICT国際戦略の推進'],
 ['n5900000.pdf', '郵政行政の推進']]

In [16]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    for path, title in links:
        cur.execute(f'INSERT INTO links (path, title, base_url) VALUES(?, ?, ?)', (path, title, base_url))

### 防衛省　防衛白書

In [17]:
url_mod = 'http://www.clearing.mod.go.jp/hakusho_data/2023/pdf/index.html'

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    base_url = cur.execute('SELECT base_url FROM sources WHERE org="防衛省" AND doc="防衛白書"').fetchone()[0]

base_url

'http://www.clearing.mod.go.jp/hakusho_data/2023/pdf'

In [18]:
resp = requests.get(url_mod)

In [19]:
from bs4.dammit import EncodingDetector
html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
html_encoding

'utf-8'

In [20]:
soup = BeautifulSoup(resp.content.decode(html_encoding))
soup.content
all_a = soup.find_all('a', href=re.compile(r'^R05\d{6}\.pdf$'))
links = [[a['href'], a.text] for a in all_a]
sorted(links)

[['R05000010.pdf', '刊行によせて'],
 ['R05000021.pdf', '激変する時代～10年の変化～'],
 ['R05000022.pdf', '国家防衛戦略'],
 ['R05000031.pdf', 'わが国を取り巻く安全保障環境'],
 ['R05000032.pdf', 'わが国の安全保障・防衛政策'],
 ['R05000033.pdf', '防衛目標を実現するための3つのアプローチ'],
 ['R05000034.pdf', '共通基盤などの強化'],
 ['R05010100.pdf', '概観'],
 ['R05010200.pdf', 'ロシアによるウクライナ侵略とウクライナによる防衛'],
 ['R05010301.pdf', '米国'],
 ['R05010302.pdf', '中国'],
 ['R05010303.pdf', '米国と中国の関係など'],
 ['R05010304.pdf', '朝鮮半島'],
 ['R05010305.pdf', 'ロシア'],
 ['R05010306.pdf', '大洋州'],
 ['R05010307.pdf', '東南アジア'],
 ['R05010308.pdf', '南アジア'],
 ['R05010309.pdf', '欧州・カナダ'],
 ['R05010310.pdf', 'その他の地域など（中東・アフリカを中心に）'],
 ['R05010401.pdf', '情報戦などにも広がりをみせる科学技術をめぐる動向'],
 ['R05010402.pdf', '宇宙領域をめぐる動向'],
 ['R05010403.pdf', 'サイバー領域をめぐる動向'],
 ['R05010404.pdf', '電磁波領域をめぐる動向'],
 ['R05010405.pdf', '海洋をめぐる動向'],
 ['R05010406.pdf', '大量破壊兵器の移転・拡散'],
 ['R05010407.pdf', '気候変動が安全保障環境や軍に与える影響'],
 ['R05020101.pdf', 'わが国の安全保障を確保する方策'],
 ['R05020102.pdf', '憲法と防衛政策の基本'],
 ['R05020103.pdf', 'わが国の安全保障政策の体系'],
 [

In [21]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    for path, title in links:
        cur.execute(f'INSERT INTO links (path, title, base_url) VALUES(?, ?, ?)', (path, title, base_url))

## Table: texts

防衛省のサイトは反応悪い。しかし、防衛省発行の白書は写真豊富で読んでいて面白い。

In [23]:
import traceback
import fitz

# Reference: https://stackoverflow.com/questions/67558627/problem-while-joining-two-url-components-with-urllib
# urljoinは使わない方が良い。スラッシュありなしで結果が異なるため。
def joinurl(baseurl, path):
    return '/'.join([baseurl.rstrip('/'), path.lstrip('/')])

extract_text = lambda page: (page.number, page.get_text("text").replace('\n', ''))

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS texts')
    cur.execute('CREATE TABLE texts (link_id INTEGER, page INTEGER, text TEXT, UNIQUE(link_id, page, text), FOREIGN KEY(link_id) REFERENCES links(id))')

    links = cur.execute('SELECT * FROM links').fetchall()
    
    total = len(links)
    cnt = 1

    for link in links:
        print(f'{cnt}/{total}', end=' ')
        cnt += 1
        link_id = link[0]
        url = joinurl(link[3], link[1])
        #print(url)
        resp = requests.get(url)
        try:
            doc = fitz.open(stream=resp.content)
            for page in doc:
                page_num, text = extract_text(page)
                cur.execute('INSERT INTO texts (link_id, page, text) VALUES (?, ?, ?)', (link_id, page_num, text))
        except:
            print(url)
            traceback.print_exc()

1/129 2/129 3/129 4/129 5/129 6/129 7/129 8/129 9/129 10/129 11/129 12/129 13/129 14/129 15/129 16/129 17/129 18/129 19/129 20/129 21/129 22/129 23/129 24/129 25/129 26/129 27/129 28/129 29/129 30/129 31/129 32/129 33/129 34/129 35/129 36/129 37/129 38/129 39/129 40/129 41/129 42/129 43/129 44/129 45/129 46/129 47/129 48/129 49/129 50/129 51/129 52/129 53/129 54/129 55/129 56/129 57/129 58/129 59/129 60/129 61/129 62/129 63/129 64/129 65/129 66/129 67/129 68/129 69/129 70/129 71/129 72/129 73/129 74/129 75/129 76/129 77/129 78/129 79/129 80/129 81/129 82/129 83/129 84/129 85/129 86/129 87/129 88/129 89/129 90/129 91/129 92/129 93/129 94/129 95/129 96/129 97/129 98/129 99/129 100/129 101/129 102/129 103/129 104/129 105/129 106/129 107/129 108/129 109/129 110/129 111/129 112/129 113/129 114/129 115/129 116/129 117/129 118/129 119/129 120/129 121/129 122/129 123/129 124/129 125/129 126/129 127/129 128/129 129/129 